In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pointbiserialr

In [2]:
df = pd.read_csv('sga_24_25_adv.csv')

# removing last row as it is an aggergate row of stats
df = df[:-1]

# df_adv = pd.read_csv('nikola_jokic_24_25_adv.csv')
# df_adv = df_adv[:-1]

In [17]:
# df = df.merge(df_adv, on='Gtm')

In [18]:
df

,Rk,Gcar,Gtm,Date,Team,Unnamed: 5,Opp,Result,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1.0,676.0,1.0,24/10/2024,DEN,NaN,OKC,"L, 87-102",*,35:12:00,...,8.0,12.0,13.0,2.0,1.0,3.0,3.0,16.0,21.7,-9.0
1,2.0,677.0,2.0,26/10/2024,DEN,NaN,LAC,"L, 104-109",*,36:46:00,...,6.0,9.0,4.0,2.0,1.0,5.0,2.0,41.0,31.2,7.0
2,3.0,678.0,3.0,28/10/2024,DEN,@,TOR,"W, 127-125 (OT)",*,43:41:00,...,7.0,10.0,4.0,1.0,2.0,3.0,2.0,40.0,33.5,9.0
3,4.0,679.0,4.0,29/10/2024,DEN,@,BRK,"W, 144-139 (OT)",*,40:31:00,...,12.0,18.0,16.0,0.0,1.0,1.0,3.0,29.0,38.1,8.0
4,5.0,680.0,5.0,1/11/2024,DEN,@,MIN,"L, 116-119",*,39:42:00,...,7.0,9.0,13.0,3.0,1.0,3.0,1.0,26.0,30.1,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,66.0,741.0,78.0,4/04/2025,DEN,@,GSW,"L, 104-118",*,37:13:00,...,10.0,12.0,9.0,2.0,0.0,6.0,2.0,33.0,32.2,-4.0
78,67.0,742.0,79.0,6/04/2025,DEN,NaN,IND,"L, 120-125",*,39:00:00,...,12.0,15.0,13.0,2.0,1.0,4.0,0.0,41.0,38.2,1.0
79,68.0,743.0,80.0,9/04/2025,DEN,@,SAC,"W, 124-116",*,37:42:00,...,9.0,12.0,11.0,4.0,0.0,1.0,2.0,20.0,28.3,9.0
80,69.0,744.0,81.0,11/04/2025,DEN,NaN,MEM,"W, 117-109",*,40:55:00,...,13.0,16.0,13.0,2.0,0.0,5.0,3.0,26.0,26.4,11.0


In [19]:
df["Result"] = df["Result"].apply(lambda x: 1 if x.startswith('W') else 0)

def convert_to_seconds(mp_str):
    try:
        if pd.isna(mp_str):
            return np.nan
        parts = mp_str.split(':')
        if len(parts) != 3:
            return np.nan  # Or raise error
        minutes, seconds, milliseconds = map(int, parts)
        total_seconds = minutes * 60 + seconds + milliseconds / 1000
        return total_seconds
    except:
        return np.nan

df['MP'] = df['MP'].apply(convert_to_seconds).astype(float)


In [20]:
filtered_df = df.dropna(subset=['GmSc'])

In [21]:
column_names = [col for col in df.columns]
skipped_cols = column_names[:9]

In [22]:
results_fillna = []
results_dropna = []


for col in column_names:
    # 1. Fill NaN with 0

    if col in skipped_cols:
        continue

    try:
        r_fill, p_fill = pointbiserialr(df['Result'], df[col].fillna(0))
        results_fillna.append((col, r_fill, p_fill))
    except Exception as e:
        results_fillna.append((col, None, None))
        print(f"[fillna] Could not calculate for {col}: {e}")

    # 2. Drop NaN values in current column
    try:
        filtered_df = df.dropna(subset=[col])
        r_drop, p_drop = pointbiserialr(filtered_df['Result'], filtered_df[col])
        results_dropna.append((col, r_drop, p_drop))
    except Exception as e:
        results_dropna.append((col, None, None))
        print(f"[dropna] Could not calculate for {col}: {e}")

# Display results
print("\n--- Correlation with 'Result' (NaNs filled with 0) ---")
for col, r, p in results_fillna:
    print(f"{col:>5}: r_pb = {r:.4f} | p = {p:.4g}" if r is not None else f"{col:>5}: Calculation failed")

print("\n--- Correlation with 'Result' (Rows with NaNs dropped) ---")
for col, r, p in results_dropna:
    print(f"{col:>5}: r_pb = {r:.4f} | p = {p:.4g}" if r is not None else f"{col:>5}: Calculation failed")


--- Correlation with 'Result' (NaNs filled with 0) ---
   MP: r_pb = 0.1693 | p = 0.1283
   FG: r_pb = 0.0841 | p = 0.4523
  FGA: r_pb = 0.0332 | p = 0.7674
  FG%: r_pb = 0.2804 | p = 0.01073
   3P: r_pb = -0.0183 | p = 0.8706
  3PA: r_pb = -0.1033 | p = 0.3558
  3P%: r_pb = 0.1970 | p = 0.07602
   2P: r_pb = 0.1057 | p = 0.3447
  2PA: r_pb = 0.0915 | p = 0.4134
  2P%: r_pb = 0.2467 | p = 0.02547
 eFG%: r_pb = 0.2647 | p = 0.01625
   FT: r_pb = 0.2093 | p = 0.05917
  FTA: r_pb = 0.1856 | p = 0.09507
  FT%: r_pb = 0.2300 | p = 0.03763
  ORB: r_pb = 0.1138 | p = 0.3088
  DRB: r_pb = 0.2586 | p = 0.019
  TRB: r_pb = 0.2426 | p = 0.02808
  AST: r_pb = 0.2804 | p = 0.01072
  STL: r_pb = 0.2491 | p = 0.02402
  BLK: r_pb = 0.1811 | p = 0.1034
  TOV: r_pb = 0.0841 | p = 0.4526
   PF: r_pb = 0.1081 | p = 0.3337
  PTS: r_pb = 0.1203 | p = 0.2817
 GmSc: r_pb = 0.2512 | p = 0.0228
  +/-: r_pb = 0.6521 | p = 3.228e-11

--- Correlation with 'Result' (Rows with NaNs dropped) ---
   MP: r_pb = -0.211

In [ ]:
# Create DataFrames from the results
df_fillna = pd.DataFrame(results_fillna, columns=['Column', 'R_0', 'p_value_0'])
df_dropna = pd.DataFrame(results_dropna, columns=['Column', 'R_removed', 'p_value_removed'])

# Merge the two DataFrames on 'Column'
merged_results = pd.merge(df_fillna, df_dropna, on='Column', how='outer')

# Save to CSV
merged_results.to_csv('jokic_correlation_results.csv', index=False)
print("✅ Results exported to 'correlation_results.csv'")

In [ ]:
df["Result"] = df["Result"].apply(lambda x: 1 if x.startswith('W') else 0)

In [ ]:
r, p = pointbiserialr(df['Result'], df['GmSc'].fillna(0))
print("r_pb:", r)
print("p-value:", p)

In [ ]:
r, p = pointbiserialr(df['Result'], df['FG'].fillna(0))
print("r_pb:", r)
print("p-value:", p)

In [ ]:
def convert_to_seconds(mp_str):
    try:
        if pd.isna(mp_str):
            return np.nan
        parts = mp_str.split(':')
        if len(parts) != 3:
            return np.nan  # Or raise error
        minutes, seconds, milliseconds = map(int, parts)
        total_seconds = minutes * 60 + seconds + milliseconds / 1000
        return total_seconds
    except:
        return np.nan

df['MP'] = df['MP'].apply(convert_to_seconds).astype(float)

r, p = pointbiserialr(df['Result'], df['MP'].fillna(0))
print("r_pb:", r)
print("p-value:", p)

In [ ]:
r, p = pointbiserialr(df['Result'], df['FGA'].fillna(0))
print("r_pb:", r)
print("p-value:", p)

In [ ]:
r, p = pointbiserialr(df['Result'], df['PTS'].fillna(0))
print("r_pb:", r)
print("p-value:", p)

In [ ]:
r, p = pointbiserialr(df['Result'], df['+/-'].fillna(0))
print("r_pb:", r)
print("p-value:", p)

In [ ]:
x = df[['+/-']].fillna(0)
y = df['Result']

model = LogisticRegression()
model.fit(x, y)

# Generate smooth curve data
x_vals = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
# x_vals = np.linspace(-100, 100, 3000).reshape(-1, 1)
# Predict probabilities for the logistic curve
y_probs = model.predict_proba(x_vals)[:, 1]  # probability of class 1

# Plot the logistic regression curve
plt.plot(x_vals, y_probs, color='blue', label='Logistic Regression')

# Plot the real data points (scatter)
# Use y values as dots at 0 or 1, jitter slightly to see overlap
plt.scatter(x, y, color='black', zorder=20, label='Real Data', alpha=0.6)

plt.xlabel('+/-')      # change to your feature name
plt.ylabel('Probability / Class')
plt.title('Logistic Regression Fit and Real Data')
plt.legend()
plt.grid(True)
plt.show()